In [3]:
import plotly
import plotly.graph_objects as go
import numpy as np
import warnings
from dev import *

warnings.filterwarnings("ignore")


aggregate_attack_name_dict = {
    "Distortions (single)": [
        "distortion_single_rotation",
        "distortion_single_resizedcrop",
        "distortion_single_erasing",
        "distortion_single_brightness",
        "distortion_single_contrast",
        "distortion_single_blurring",
        "distortion_single_noise",
        "distortion_single_jpeg",
    ],
    "Distortions (combo)": [
        "distortion_combo_geometric",
        "distortion_combo_photometric",
        "distortion_combo_degradation",
        "distortion_combo_all",
    ],
    "Embedding Attack (grey-box)": ["adv_emb_same_vae_untg"],
    "Embedding Attack (black-box)": [
        "adv_emb_clip_untg_alphaRatio_0.05_step_200",
        "adv_emb_sdxl_vae_untg",
        "adv_emb_klf16_vae_untg",
    ],
    "Surrogate Detector Attack": ["adv_cls_wm1_wm2_0.01_50_warm"],
    "Regeneration (black-box)": ["regen_diffusion", "kl_vae"],
    "Rinsing regeneration (black-box)": ["2x_regen", "4x_regen"],
    "Mixed regeneration (black-box)": ["4x_regen_bmshj", "4x_regen_kl_vae"],
}


result_dict = {k: {} for k in WATERMARK_METHODS.keys()}
quality_dict = {k: {} for k in aggregate_attack_name_dict.keys()}
for source_name in WATERMARK_METHODS.keys():
    for aggregate_attack_name, attack_names in aggregate_attack_name_dict.items():
        performances = []
        qualities = []
        for attack_name in attack_names:
            json_dict = get_all_json_paths(
                lambda _dataset_name, _attack_name, _attack_strength, _source_name, _result_type: (
                    _source_name == source_name and _attack_name == attack_name
                )
            )
            record_keys = set(
                [(key[0], key[3], key[1], key[2]) for key in json_dict.keys()]
            )
            for dataset_name, source_name, attack_name, attack_strength in record_keys:
                performances.append(
                    get_performance(
                        dataset_name,
                        source_name,
                        attack_name,
                        attack_strength,
                        mode="removal",
                    )["low1000_1"]
                )
                qualities.append(
                    get_quality(
                        dataset_name,
                        source_name,
                        attack_name,
                        attack_strength,
                        mode="removal",
                    ).get("psnr", None)
                )
        result_dict[source_name][aggregate_attack_name] = np.array(
            [p for p in performances if p is not None]
        ).mean()
        quality_dict[aggregate_attack_name] = np.array(
            [q for q in qualities if q is not None]
        ).mean()


fig = go.Figure()

# Retrieve the default Plotly colors
default_colors = plotly.colors.DEFAULT_PLOTLY_COLORS

alpha = 0.2  # Set your desired alpha value here, e.g., 0.5 for 50% transparency
modified_colors = [
    color.replace("rgb", "rgba").replace(")", f", {alpha})") for color in default_colors
]

for index, source_name in enumerate(WATERMARK_METHODS.keys()):
    r = list(result_dict[source_name].values())
    r.append(r[0])
    theta = [
        f" {k} (PSNR={quality_dict[k]:.1f})" for k in aggregate_attack_name_dict.keys()
    ]
    theta.append(theta[0])
    fig.add_trace(
        go.Scatterpolar(
            r=r,
            theta=theta,
            fill="toself",
            name=WATERMARK_METHODS[source_name],
            fillcolor=modified_colors[index],
        )
    )

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])), showlegend=True
)

fig.show()

In [5]:
import plotly
import plotly.graph_objects as go
import numpy as np
import warnings
from dev import *

warnings.filterwarnings("ignore")


aggregate_attack_name_dict = {
    "Distortions (single)": [
        "distortion_single_rotation",
        "distortion_single_resizedcrop",
        "distortion_single_erasing",
        "distortion_single_brightness",
        "distortion_single_contrast",
        "distortion_single_blurring",
        "distortion_single_noise",
        "distortion_single_jpeg",
    ],
    "Distortions (combo)": [
        "distortion_combo_geometric",
        "distortion_combo_photometric",
        "distortion_combo_degradation",
        "distortion_combo_all",
    ],
    "Embedding Attack (grey-box)": ["adv_emb_same_vae_untg"],
    "Embedding Attack (black-box)": [
        "adv_emb_clip_untg_alphaRatio_0.05_step_200",
        "adv_emb_sdxl_vae_untg",
        "adv_emb_klf16_vae_untg",
    ],
    "Surrogate Detector Attack": ["adv_cls_wm1_wm2_0.01_50_warm"],
    "Regeneration (black-box)": ["regen_diffusion", "kl_vae"],
    "Rinsing regeneration (black-box)": ["2x_regen", "4x_regen"],
    "Mixed regeneration (black-box)": ["4x_regen_bmshj", "4x_regen_kl_vae"],
}


result_dict = {k: {} for k in WATERMARK_METHODS.keys()}
quality_dict = {k: {} for k in aggregate_attack_name_dict.keys()}
for source_name in WATERMARK_METHODS.keys():
    for aggregate_attack_name, attack_names in aggregate_attack_name_dict.items():
        performances = []
        qualities = []
        for attack_name in attack_names:
            json_dict = get_all_json_paths(
                lambda _dataset_name, _attack_name, _attack_strength, _source_name, _result_type: (
                    _source_name == source_name and _attack_name == attack_name
                )
            )
            record_keys = set(
                [(key[0], key[3], key[1], key[2]) for key in json_dict.keys()]
            )
            for dataset_name, source_name, attack_name, attack_strength in record_keys:
                performances.append(
                    get_performance(
                        dataset_name,
                        source_name,
                        attack_name,
                        attack_strength,
                        mode="removal",
                    )["low1000_1"]
                )
                qualities.append(
                    get_quality(
                        dataset_name,
                        source_name,
                        attack_name,
                        attack_strength,
                        mode="removal",
                    ).get("psnr", None)
                )
        result_dict[source_name][aggregate_attack_name] = np.array(
            [p for p in performances if p is not None]
        ).min()
        quality_dict[aggregate_attack_name] = np.array(
            [q for q in qualities if q is not None]
        ).mean()


fig = go.Figure()

# Retrieve the default Plotly colors
default_colors = plotly.colors.DEFAULT_PLOTLY_COLORS

alpha = 0.2  # Set your desired alpha value here, e.g., 0.5 for 50% transparency
modified_colors = [
    color.replace("rgb", "rgba").replace(")", f", {alpha})") for color in default_colors
]

for index, source_name in enumerate(WATERMARK_METHODS.keys()):
    r = list(result_dict[source_name].values())
    r.append(r[0])
    theta = [
        f" {k} (PSNR={quality_dict[k]:.1f})" for k in aggregate_attack_name_dict.keys()
    ]
    theta.append(theta[0])
    fig.add_trace(
        go.Scatterpolar(
            r=r,
            theta=theta,
            fill="toself",
            name=WATERMARK_METHODS[source_name],
            fillcolor=modified_colors[index],
        )
    )

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])), showlegend=True
)

fig.show()